# Intro to Natural Language Processing 

> "You shall know a word by the company it keeps." ~ John R. Firth

In [127]:
%%html

<iframe width="768" height="432" src="https://miro.com/app/live-embed/uXjVOlC3sTw=/?moveToViewport=-1354,-1121,2108,1681&embedId=334819522676" frameborder="0" scrolling="no" allowfullscreen></iframe>

## Goal

The goal of this short demo is to cover the process of preparing and transforming text data in order to build a similarity based recommender system.

## Table of Contents

1. Libraries
2. The Data
3. Flash NLP Intro
4. Cleaning
5. Recommendation System
6. Summary

## 1. Libraries

Download the following libraries, if not available already. You can check with `!pip list` or with `!conda list` in a new cell.

In [ ]:
# !pip install -U spacy panel

In [139]:
import json, re, spacy
from random import choice
import pandas as pd, numpy as np
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import panel as pn
from concurrent.futures import ProcessPoolExecutor

pn.extension()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. The Data

With have been given a random corpus of news articles, plus some additional information (outlined below), and we want to make a useful product with it.

| Column | Content |
|--------|---------|
|title |Title of article|
|text | Text inside article|
|domain | Domain Url of article|
|date | YYYY-MM-DD Time|
|description | Abstract of article|
|url | Url of article|
|image_url | Image if available|

Here is the full description of the dataset from HugginFace.

> "CC-News dataset contains news articles from news sites all over the world. The data is available on AWS S3 in the Common Crawl bucket at /crawl-data/CC-NEWS/. This version of the dataset has been prepared using news-please - an integrated web crawler and information extractor for news.
It contains 708241 English language news articles published between Jan 2017 and December 2019. It represents a small portion of the English language subset of the CC-News dataset." ~ [Hugging Face cc_news](https://huggingface.co/datasets/cc_news)

Before we do any data cleaning, let's read in the data and explore it a bit.

In [129]:
df = pd.read_parquet("cc_news_sample.parquet")

Let's see how many articles we have and then examine the columns.

In [130]:
df.shape

(5000, 7)

In [132]:
df.head()

,title,text,domain,date,description,url,image_url
0,Cristiano Ronaldo banned for 5 games after pus...,(AP Photo/Manu Fernandez). Real Madrid's Crist...,www.wave3.com,2017-08-14 00:00:00,,http://www.wave3.com/story/36129496/cristiano-...,http://APMOBILE.images.worldnow.com/images/146...
1,Why the Philippines is not truly independent,"FOR leftists, American assistance in the Maraw...",www.manilatimes.net,2017-06-15 00:21:39,,http://www.manilatimes.net/philippines-not-tru...,http://manilatimes.net/wp-content/uploads/2016...
2,Tyron Charles death: Murder victim 'dumped in ...,Image copyright Family handout Image caption T...,www.bbc.com,2018-07-04 20:09:54,Police secretly recorded prison visits to find...,https://www.bbc.com/news/uk-england-leeds-4471...,https://ichef.bbci.co.uk/news/1024/branded_new...
3,North Korean official accuses U.S. of turning ...,Justice Neil Gorsuch heard his first arguments...,theweek.com,2017-04-17 19:12:38,"Official site of The Week Magazine, offering c...",http://theweek.com/speedreads/692792/north-kor...,http://api.theweek.com/sites/default/files/sty...
4,Spring's Sweet Start: Dairy Queen's Free Cone Day,"What to Know Tuesday, March 20\nParticipating ...",www.nbclosangeles.com,2018-03-19 10:44:02,Find your pay-nothing small vanilla cone on th...,https://www.nbclosangeles.com/news/local/Sprin...,https://media.nbclosangeles.com/images/1200*67...


## 3. Flash NLP Intro

Let's pick a random article using `.loc[index, column]` on our dataframe and let's examine it.

In [144]:
random_article = df.iloc[choice(range(5000)), 1]
pprint(random_article)

('British foreign secretary Boris Johnson has denounced Moscow’s denials of '
 'involvement in the nerve weapon attack on a former Russian double agent as '
 '“increasingly absurd”.\n'
 'Mr Johnson’s remarks came as he received further support from the European '
 'Union and Nato on Monday over the attack.\n'
 'The secretary of state for foreign affairs discussed the attempted murder of '
 'Sergei Skripal and his daughter Yulia at a session of EU foreign ministers, '
 'which he addressed. Mr Johnson also discussed the incident in a meeting with '
 'Nato secretary-general Jens Stoltenberg after statements of solidarity last '
 'week by a host of Western governments.\n'
 '“Our Nato allies have shown such undivided support,” said Mr Johnson after '
 'meeting Mr Stoltenberg.\n'
 'All 28 EU foreign ministers issued a joint statement on the attack, '
 'expressing “unqualified solidarity”.\n'
 '“The European Union takes extremely seriously the UK government’s assessment '
 'that it is highly 

Notice how the review above looks a bit odd and it has a few characters that will not be useful for our analysis. Let's examine a cleaner version of the article above by running it through `spaCy`'s tokenizer.

When we tokenize a document, we are separating all of its content into each of its components, i.e. words, numbers, punctiations and the like, to make it easier to process it, clean it, transform it and to run computations on it.

For this part, we will load an english model, instantiate it and pass an example article through it. You may need to run the cell below first to download the english model.

In [ ]:
# !python -m spacy download en_core_web_lg

In [145]:
nlp = spacy.load("en_core_web_lg")

In [146]:
parsed_article = nlp(random_article)

In [147]:
parsed_article

British foreign secretary Boris Johnson has denounced Moscow’s denials of involvement in the nerve weapon attack on a former Russian double agent as “increasingly absurd”.
Mr Johnson’s remarks came as he received further support from the European Union and Nato on Monday over the attack.
The secretary of state for foreign affairs discussed the attempted murder of Sergei Skripal and his daughter Yulia at a session of EU foreign ministers, which he addressed. Mr Johnson also discussed the incident in a meeting with Nato secretary-general Jens Stoltenberg after statements of solidarity last week by a host of Western governments.
“Our Nato allies have shown such undivided support,” said Mr Johnson after meeting Mr Stoltenberg.
All 28 EU foreign ministers issued a joint statement on the attack, expressing “unqualified solidarity”.
“The European Union takes extremely seriously the UK government’s assessment that it is highly likely that the Russian Federation is responsible,” the statement n

Notice how much nicer our article looks now.

We can also grab the sentences and view them one by one using the attribute `.sents` and the built in python function `next()`, since the attribute of a document that has been tokenized by spacy will always return an iterator. Conversely, we can add it to a loop and show each of the sentences in an article.

In [148]:
next(enumerate(parsed_article.sents))

(0,
 British foreign secretary Boris Johnson has denounced Moscow’s denials of involvement in the nerve weapon attack on a former Russian double agent as “increasingly absurd”.)

In [149]:
for num, sentence in enumerate(parsed_article.sents):
    print(f"Sentence #{num}:\n {sentence}\n")

Sentence #0:
 British foreign secretary Boris Johnson has denounced Moscow’s denials of involvement in the nerve weapon attack on a former Russian double agent as “increasingly absurd”.

Sentence #1:
 
Mr Johnson’s remarks came as he received further support from the European Union and Nato on Monday over the attack.

Sentence #2:
 
The secretary of state for foreign affairs discussed the attempted murder of Sergei Skripal and his daughter Yulia at a session of EU foreign ministers, which he addressed.

Sentence #3:
 Mr Johnson also discussed the incident in a meeting with Nato secretary-general Jens Stoltenberg after statements of solidarity last week by a host of Western governments.

Sentence #4:
 
“Our Nato allies have shown such undivided support,” said Mr Johnson after meeting Mr Stoltenberg.

Sentence #5:
 
All 28 EU foreign ministers issued a joint statement on the attack, expressing “unqualified solidarity”.

Sentence #6:
 
“The European Union takes extremely seriously the UK 

We can also have a look at the different kinds of entities in an article. These entities can be a person (called PERSON), and number (called CARDINAL), a geopolitical entity (called GPE), etc.

In [150]:
for num, entity in enumerate(parsed_article.ents):
    print(f"Entity #{num}: {entity} -- {entity.label_}\n")

Entity #0: British -- NORP

Entity #1: Boris Johnson -- PERSON

Entity #2: Moscow -- GPE

Entity #3: Russian -- NORP

Entity #4: Johnson -- PERSON

Entity #5: the European Union -- ORG

Entity #6: Nato -- ORG

Entity #7: Monday -- DATE

Entity #8: Sergei Skripal -- PERSON

Entity #9: Yulia -- PERSON

Entity #10: EU -- ORG

Entity #11: Johnson -- PERSON

Entity #12: Nato -- ORG

Entity #13: Jens Stoltenberg -- PERSON

Entity #14: solidarity -- ORG

Entity #15: last week -- DATE

Entity #16: Western -- NORP

Entity #17: Nato -- ORG

Entity #18: Johnson -- PERSON

Entity #19: Stoltenberg -- PERSON

Entity #20: 28 -- CARDINAL

Entity #21: EU -- ORG

Entity #22: The European Union -- ORG

Entity #23: UK -- GPE

Entity #24: the Russian Federation -- GPE

Entity #25: Russia -- GPE

Entity #26: Vladimir Putin -- PERSON

Entity #27: Johnson -- PERSON

Entity #28: Russian -- NORP

Entity #29: EU -- ORG

Entity #30: a day -- DATE

Entity #31: Putin -- PERSON

Entity #32: six-year -- DATE

Entity 

In [151]:
spacy.explain("LOC")

'Non-GPE locations, mountain ranges, bodies of water'

We can also check weather a word is a stopword or a punctuation, or we can even lemmatize our articles. Lemmatization is a way of taking the root of a word and bringing similar words to a common denominator, for example, `was` will become `be` and most plural words will become singular words.

In [152]:
new_list = []

for token in parsed_article:
    new_list.append(token.text)
    
    
new_list[:10]

['British',
 'foreign',
 'secretary',
 'Boris',
 'Johnson',
 'has',
 'denounced',
 'Moscow',
 '’s',
 'denials']

In [153]:
new_list = [token.text for token in parsed_article]

new_list[:10]

['British',
 'foreign',
 'secretary',
 'Boris',
 'Johnson',
 'has',
 'denounced',
 'Moscow',
 '’s',
 'denials']

In [154]:
# here we are taking out of the parsed article each token
token_text = [token.text for token in parsed_article]

# here we are lemmatizing each word possible
token_lemmas = [token.lemma_ for token in parsed_article]

# stopwords are very common so here we will extract a variable that will tell us whether
# a token is a stopword or not
token_stop = [token.is_stop for token in parsed_article]

# a token is a pinctuation or not
token_punc = [token.is_punct for token in parsed_article]

# we will now add all three to a dataframe and display it without assigning it to a variable
pd.DataFrame(zip(token_text, token_lemmas, token_punc, token_stop), columns=['Original Text', 'Lemmatized Text', 'Punctuations', 'stopwords']).head(50)

,Original Text,Lemmatized Text,Punctuations,stopwords
0,British,british,False,False
1,foreign,foreign,False,False
2,secretary,secretary,False,False
3,Boris,Boris,False,False
4,Johnson,Johnson,False,False
5,has,have,False,True
6,denounced,denounce,False,False
7,Moscow,Moscow,False,False
8,’s,’s,False,True
9,denials,denial,False,False


## 4. Preparation

Let's start by checking if our dataset contains any missin values, and then evaluate the amount of memory we are currently using from our machine.

In [ ]:
df.isna().sum()

In [ ]:
df.info(memory_usage='deep')

Depending on the random sample you choose at the beginning, you may or may not have a lot. If so, getting rid of the columns you don't need will help release some of the memory in your machine.

In [ ]:
df.drop(['url', 'image_url', 'domain'], axis=1, inplace=True)

Perfect! Let's now extract the `text` column and normalize it. This means we will use `spacy` to,
- take out anything that is not a word or a number,
- convert to lower case,
- strip the spaces around the words,
- tokenize the articles,
- remove stopwords (we will use spaCy's list of stopwords for this),
- and then join the cleaned tokens back together.

In [ ]:
articles = df['text'].values

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
len(STOP_WORDS), STOP_WORDS

In [ ]:
def normalize_doc(doc):
    """
    This function normalizes your list of documents by taking only
    words, numbers, and spaces in between them. It then filters out
    stop words.
    """
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = nlp(doc)
    filtered_tokens = [token.lemma_ for token in tokens if not token.is_stop]
    doc = ' '.join(filtered_tokens).replace(" \n ", "")
    return doc

In [ ]:
random_article

In [ ]:
normalize_doc(random_article)

Since we have quite a few articles, this operation can take quite some time unless we do the cleaning process concurrently or in parallel. We will do this using the `ProcessPoolExecutor()` from the `concurrent.futures` module.

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=12) as e:
    processed_articles = list(e.map(normalize_doc, articles))

We will add the cleaned versions of the documents back into the dataframe and loop over these while taking the lenght (in characters terms) of each article.

In [ ]:
%%time

df['clean_text'] = processed_articles
df['len_clean_text'] = df['clean_text'].apply(len)
df['len_dirty_text'] = df['text'].apply(len)

Let's now save our cleaned dataset in case we need to restart our notebook and begin the analysis again. We will also release a bit of memory by getting rid of all the data and variables we have loaded up since the beginning of the notebook.

In [ ]:
df.head(2)

In [ ]:
df = df[['title', 'date', 'clean_text', 'len_clean_text', 'len_dirty_text']].reset_index(drop=True)

It wouldn't make any sense to feed to our algorithms articles with a tiny amount of characters, so let's examine the distribution of characters among both, the raw and the clean version of our articles.

In [ ]:
df[['len_clean_text', 'len_dirty_text']].describe().T

In [ ]:
df[['len_clean_text', 'len_dirty_text']].skew()

![img](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn.analyticsvidhya.com%2Fwp-content%2Fuploads%2F2020%2F06%2Fsk1.png&f=1&nofb=1)

Now that we know we have a skewed distribution of characters, let's fix that by setting up a rule. We'll evaluate an article using the tweets' maximum character count of 280, at the time of writing, and filter out all articles with less than that. Let's check how many we have first.

In [ ]:
greater_than_a_tweet = df['len_clean_text'] > 280
greater_than_a_tweet.sum()

In [ ]:
df = df[greater_than_a_tweet].copy()

In [ ]:
df.shape

# 5. Recommendation System

Recommendation systems can come in many different forms and sizes. We can create a system that takes into account the behaviour of other users, or a system that only looks at similar articles or items to make a recommendation. Both are powerful systems and could cover an entire section of a book in their own right, which is why we will focus on the latter category, the one that makes recommendations based on similar articles.

To create our recommendation system we first need to convert our articles into a numerical representation. We do this with a so-called bag of words (bow). BOWs are matrices with the documents in the rows, the terms contained in all documents along the columns. The frequency with which each term appears in each document along the values can be found in the doc-token combination. To create this kind of representation we can use `sklearn`'s `CountVectorizer` or `TfidfVectorizer` classes. The latter being the normalized version of the former, i.e. the frequency of a word divided by the amount of documents in which it appears.

To use this classes we first instantiate them, fit the data to them so that they can learn the vocabulary of our corpus, and then we tranform the corpus into a sparse matrix. These sparse matrices hold the location of all non-zero values to make it easier to store the data and compute on it.

In [ ]:
%%time

# we first instantiate our class
tf = TfidfVectorizer(min_df=0.035, max_df=0.80)

# we can fit and transform the data in the same step
tfidf_matrix = tf.fit_transform(df['clean_text'].values)

# evaluate the shape of our matrix
tfidf_matrix.shape

We can access our vocabulary with `.get_feature_names()` method.

In [ ]:
tf.get_feature_names()[500:550]

The next step is to get the distance between documents and words to see how close and how far, based on words only, are two documents from one another. The `cosine_similarity` function we imported earlier can do this for us, and afterwards, we can create a dataframe to evaluate our results.

**Note:** this operation can take a few minutes if you are using the entire dataset. Make sure to grab some ☕️ 😎

In [ ]:
%%time

doc_sim = cosine_similarity(tfidf_matrix)

In [ ]:
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

In [ ]:
doc_sim.shape

The reason we see a X000xX000 matrix is because both halfs alonside the diagonal line are identical, hence, we have the similarity of all docs vs all docs.

In [ ]:
articles_list = df['title'].values
articles_list.shape, articles_list

Let's now
1. pick a title at random
2. get the index of such title
3. select the corresponding row for such title in our new document similarity dataframe
4. sort the index of such values
5. return the top 5 article titles

In [ ]:
from random import choice

In [ ]:
a_title = choice(articles_list)
a_title

In [ ]:
article_idx = np.where(articles_list == a_title)[0][0]
article_idx

In [ ]:
article_similarities = doc_sim_df.iloc[article_idx].values
article_similarities

In [ ]:
# note that we don't select the first one as this should always be one
similar_articles_idxs = np.argsort(-article_similarities)[1:10]
similar_articles_idxs

In [ ]:
df.head()

In [ ]:
doc1 = nlp(df.iloc[1, 2])
doc2 = nlp(df.iloc[2, 2])

In [ ]:
doc1.similarity(doc2)

In [ ]:
a_title

In [ ]:
similar_articles = articles_list[similar_articles_idxs]
pprint(similar_articles.tolist())

Lastly, we will create create a mini-dashboard containing,
1. a widget with all of our titles,
2. a function with the steps we followed above,
3. a panel object to store a title, the widget, and the function.

In [ ]:
titles = df.title.unique().tolist()
title_widget = pn.widgets.Select(value=choice(titles), options=titles, name='Articles')

In [ ]:
@pn.depends(title_widget.param.value)
def article_recommender(title_widget):
    
    article_idx = np.where(articles_list == title_widget)[0][0]
    article_similarities = doc_sim_df.iloc[article_idx].values
    similar_title_idxs = np.argsort(-article_similarities)[1:6]
    similar_titles = articles_list[similar_title_idxs]
    
    return pn.Column(*similar_titles, width=600)

In [ ]:
text = pn.pane.Markdown(f"# Small Recommendation Engine", style={"color": "#000000"}, width=600, height=50,
                        sizing_mode="stretch_width", margin=(10,10,10,5))

In [ ]:
pn.Column(text, title_widget, article_recommender, align='center', width=600, height=300)

## 6. Summary

Blind Spots

With additional time we could have,
1. Further tweak the parameters of the vectorizers and models;
2. Create visualizations of the document similarity to find more interesting patters;
3. Take the title of an article out of the body of the article to create a better, less biased representation of the words within a document;
4. Using Pytorch's nn.CosineSimilarity would help a lot with increasing the efficiency of our recommendation system;
5. There should have been a lemmatization step in the preprocessing stage.

Takeaways,
1. Recommendation systems are an example of unsupervised machine learning;
2. Recommendation systems can be created with or without users behavioural data;
3. Creating bags of words requires careful attention to the parameters;
4. Where possible, showcase a model or system in a mini-dashboard or data visualization.